In [6]:
import numpy as np
import os
import pandas as pd
from graph_traffic.custom_funcs import  make_stable_values_null, rows_no_change
from graph_traffic.config import data_path
öo
traffic_path = os.path.join(data_path, "03-by-location", "traffic")
meteo_path = os.path.join(data_path, "03-by-location", "meteo")


tmagns = ['intensidad', 'ocupacion', 'vmed']
mmagns = ['temperatura', 'humedad_relativa', 'presion_barometrica', 'radiacion_solar',
          'precipitacion', 'vientox', 'vientoy']

mapping = pd.read_csv(os.path.join(data_path, '03-by-location', 'id_mapping.csv'))

traffic_files = os.listdir(traffic_path)
n_traffic_files = len(traffic_files)

for i, t_file in enumerate(traffic_files):
    print(f"{i+1}/{n_traffic_files}", end="\r")
    t_location = t_file.split(".")[0]
    # id de la estación meteorológica correspondiente a cada magnitud
    ids_m = mapping[mapping.id_t == int(t_location)].iloc[0][[f'id_{magn}' for magn in mmagns]].astype(int)

    # Leer los datos de tráfico
    dft = pd.read_csv(f'{traffic_path}/{t_location}.csv', parse_dates=['fecha'], index_col='fecha')
    if dft.empty:
        continue
    # Para cada estación, leemos los datos necesarios
    dfm = {estacion: pd.read_csv(f'{meteo_path}/estacion-{estacion:.0f}.csv', parse_dates=['fecha'], index_col='fecha') for estacion in ids_m.unique()}

    # Si hay más de 4 filas sin cambio, damos el valor por nulo
    dft[tmagns] = dft[tmagns].apply(make_stable_values_null, nrows=4)
    for estacion, dfmi in dfm.items():
        nm = dfmi[mmagns].apply(rows_no_change)
        for m in mmagns:
            if m in ['precipitacion', 'radiacion_solar', 'presion_barometrica']:
                continue
            dfmi[m] = np.where((nm[m]>4) & (dfmi[m]!=0), np.nan, dfmi[m])
        dfm[estacion] = dfmi

    # Hacer el merge de todas las variables meteorológicas con el tráfico
    df = dft
    for m in mmagns:
        df = df.merge(dfm[ids_m[f"id_{m}"]][[m]],
                     left_index=True, right_index=True,
                    how='left')

    #del dft, dfm
    df = df.sort_index()
    #df[mmagns] = df[mmagns].fillna(method="ffill", limit=4)
    df[mmagns] = df[mmagns].interpolate(method="linear", limit=4)

    cal = pd.read_csv(data_path+'/03-by-location/calendario.csv', sep=';')
    cal['fecha'] = pd.to_datetime(cal['Día'], dayfirst=True)
    cal['laborable / festivo / domingo festivo'] = cal['laborable / festivo / domingo festivo'].str.lower()
    festivos = cal[cal['laborable / festivo / domingo festivo']=='festivo'].fecha.dt.date.unique()

    df['dia_semana'] = df.index.day_of_week
    df.loc[df.dia_semana < 5, 'tipo_dia'] = 'laborable'
    df.loc[df.dia_semana == 5, 'tipo_dia'] = 'sabado'
    df.loc[(df.dia_semana == 6 ) | (pd.Series(df.index.date).isin(festivos)), 'tipo_dia'] = 'domingo/festivo'
    df = df.merge(df.tipo_dia.str.get_dummies(), left_index=True, right_index=True, how="left")
    del df['dia_semana'], df['tipo_dia']
    df.loc["2019":].to_csv(f"{data_path}/04-traffic-meteo-merged/{t_location}.csv")

    # timestamp_s = df.index.map(pd.Timestamp.timestamp)
    #
    # day = 24*60*60
    # year = (365.2425)*day
    #
    # df['dia_sen'] = np.sin(timestamp_s * (2 * np.pi / day))
    # df['dia_cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    # df['ano_sen'] = np.sin(timestamp_s * (2 * np.pi / year))
    # df['ano_cos'] = np.cos(timestamp_s * (2 * np.pi / year))

df.head()

,id,intensidad,ocupacion,vmed,temperatura,humedad_relativa,presion_barometrica,radiacion_solar,precipitacion,vientox,vientoy,domingo/festivo,laborable,sabado
fecha,,,,,,,,,,,,,,
2019-02-01 00:00:00,9999,10.0,NaN,NaN,6.0,96.00,929.00,0.0,0.3,-1.593149,-1.290107,0,1,0
2019-02-01 00:15:00,9999,10.0,NaN,NaN,6.0,96.25,928.75,0.0,0.5,-1.644327,-1.248437,0,1,0
2019-02-01 00:30:00,9999,9.0,NaN,NaN,6.0,96.50,928.50,0.0,0.7,-1.695506,-1.206768,0,1,0
2019-02-01 00:45:00,9999,11.0,NaN,NaN,6.0,96.75,928.25,0.0,0.9,-1.746684,-1.165098,0,1,0
2019-02-01 01:00:00,9999,10.0,NaN,NaN,6.0,97.00,928.00,0.0,1.1,-1.797862,-1.123429,0,1,0


In [4]:
df

,id,intensidad,ocupacion,vmed,temperatura,humedad_relativa,presion_barometrica,radiacion_solar,precipitacion,vientox,vientoy,domingo/festivo,laborable,sabado
fecha,,,,,,,,,,,,,,
2019-01-01 03:45:00,10000,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
2019-01-01 07:00:00,10000,18.0,NaN,NaN,-1.500000,97.000000,959.0,0.000000,0.0,0.000000,-0.000000,0,1,0
2019-01-01 07:15:00,10000,18.0,NaN,NaN,0.077778,90.222222,959.0,47.666667,0.0,-0.112902,-0.009878,0,1,0
2019-01-01 08:30:00,10000,15.0,NaN,NaN,1.655556,83.444444,959.0,95.333333,0.0,-0.225804,-0.019755,0,1,0
2019-01-01 08:45:00,10000,15.0,NaN,NaN,3.233333,76.666667,959.0,143.000000,0.0,-0.338706,-0.029633,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 03:30:00,10000,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
2020-12-31 03:45:00,10000,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
2020-12-31 10:30:00,10000,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0


In [23]:
t_location

'10001'

In [25]:
t_file

'10001.csv'